In [1]:
import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats
import pywt

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from btgym.a3c import Launcher, BaseLSTMPolicy

import tensorflow as tf
import tensorflow.contrib.rnn as rnn

In [ ]:
class Pooler():
    def __init__(self, x, window, stride, pooling_type='MAX', padding='SAME'):
        """_____"""

        self.out = tf.nn.pool(
            input=x,
            window_shape=window,
            pooling_type=pooling_type,
            padding=padding,
            dilation_rate=None,
            strides=stride,
        )
    

In [42]:
class LSTMPolicy1D(BaseLSTMPolicy):
    """
    +++++
    """
    def __init__(self, ob_space, ac_space, num_filters=32, filter_size=3, stride=2,
                 lstm_class=rnn.BasicLSTMCell, lstm_layers=(256,)):
        
        self.x = x = tf.placeholder(tf.float32, [None] + list(ob_space), name='x_in_pl')
        
        #print('GOT HERE 0')
        #print ('x:', x.shape)
        
        #smooth = Pooler(x, [3],[2],'MAX', 'SAME')
        
        #x = smooth.out
    
        # Conv layers:
        for i in range(4):
            x = tf.nn.elu(self.conv1d(x, num_filters, "l{}".format(i + 1), filter_size, stride))
        
        #print('GOT HERE 1')
        #print('x_convolved:', x)
        
        # Run LSTM along rollout time dimension and evrything else:
        super(LSTMPolicy1D, self).__init__(x, ob_space, ac_space, lstm_class, lstm_layers)
        
        
    def conv1d(self, x, num_filters, name, filter_size=3, stride=2, pad="SAME", dtype=tf.float32,
               collections=None):
        with tf.variable_scope(name):
            stride_shape =  stride
            
            #print('stride_shape:',stride_shape)
            
            filter_shape = [filter_size, int(x.get_shape()[-1]), num_filters]
            
            #print('filter_shape:', filter_shape)
            
            # there are "num input feature maps * filter height * filter width"
            # inputs to each hidden unit
            fan_in = np.prod(filter_shape[:2])
            
            # each unit in the lower layer receives a gradient from:
            # "num output feature maps * filter height * filter width" /
            #   pooling size
            fan_out = np.prod(filter_shape[:1]) * num_filters

            # initialize weights with random weights
            w_bound = np.sqrt(6. / (fan_in + fan_out))

            w = tf.get_variable("W", filter_shape, dtype, tf.random_uniform_initializer(-w_bound, w_bound),
                                collections=collections)
            b = tf.get_variable("b", [1, 1, num_filters], initializer=tf.constant_initializer(0.0),
                                collections=collections)
            return tf.nn.conv1d(x, w, stride_shape, pad) + b
   

In [43]:
class LSTMPolicy1dMulti(BaseLSTMPolicy):
    """
    +++++
    """
    def __init__(self, ob_space, ac_space, num_filters=32, filter_size=3, stride=2,
                 lstm_class=rnn.BasicLSTMCell, lstm_layers=(256,)):
        
        self.x = x = tf.placeholder(tf.float32, [None] + list(ob_space), name='x_in_pl')
        
        #print('GOT HERE 0')
        #print ('x:', x.shape)
        
        num_channels = int(x.get_shape()[-1])
        
        x_scaled = []
        for channel in range(num_channels):
            with tf.variable_scope('ch_{}'.format(channel + 1)):
                z = x[..., channel][..., None]
                # Conv layers:
                for i in range(4):
                    z = tf.nn.elu(self.conv1d(z, num_filters, "l{}".format(i + 1), filter_size, stride))
                x_scaled += [z]
                
        x_scaled = tf.concat(x_scaled, axis=-1)
        #print('GOT HERE 1')
        #print('x_convolved:', x)
        
        # Run LSTM along rollout time dimension and evrything else:
        super(LSTMPolicy1dMulti, self).__init__(x_scaled, ob_space, ac_space, lstm_class, lstm_layers)
        
        
    def conv1d(self, x, num_filters, name, filter_size=3, stride=2, pad="SAME", dtype=tf.float32,
               collections=None):
        with tf.variable_scope(name):
            stride_shape =  stride
            
            #print('stride_shape:',stride_shape)
            
            filter_shape = [filter_size, int(x.get_shape()[-1]), num_filters]
            
            #print('filter_shape:', filter_shape)
            
            # there are "num input feature maps * filter height * filter width"
            # inputs to each hidden unit
            fan_in = np.prod(filter_shape[:2])
            
            # each unit in the lower layer receives a gradient from:
            # "num output feature maps * filter height * filter width" /
            #   pooling size
            fan_out = np.prod(filter_shape[:1]) * num_filters

            # initialize weights with random weights
            w_bound = np.sqrt(6. / (fan_in + fan_out))

            w = tf.get_variable("W", filter_shape, dtype, tf.random_uniform_initializer(-w_bound, w_bound),
                                collections=collections)
            b = tf.get_variable("b", [1, 1, num_filters], initializer=tf.constant_initializer(0.0),
                                collections=collections)
            return tf.nn.conv1d(x, w, stride_shape, pad) + b

In [44]:
#tf.reset_default_graph()
#pi = LSTMPolicy1D((10,2),(3))

In [47]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy, self).__init__(**kwargs)
        self.order_penalty = 0
        self.trade_just_closed = False
        self.trade_result = None
        self.episode_result = 0
        
        self.data.sma_4 = btind.SimpleMovingAverage(self.datas[0], period=4)
        self.data.sma_8 = btind.SimpleMovingAverage(self.datas[0], period=8)
        self.data.sma_16 = btind.SimpleMovingAverage(self.datas[0], period=16)
        self.data.sma_32 = btind.SimpleMovingAverage(self.datas[0], period=32)
        self.data.sma_64 = btind.SimpleMovingAverage(self.datas[0], period=64)
        self.data.sma_128 = btind.SimpleMovingAverage(self.datas[0], period=128)
        self.data.sma_256 = btind.SimpleMovingAverage(self.datas[0], period=256)
        
        self.data.sma_20 = btind.SimpleMovingAverage(self.datas[0], period=20)
        
        self.data.dim_sma = btind.SimpleMovingAverage(
            self.datas[0],
            period=(256 + self.p.state_shape['raw_state'].shape[0])
        )
        self.data.dim_sma.plotinfo.plot = False
        
    def notify_trade(self, trade):    
        if trade.isclosed:
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
        
    def __get_state(self):
        T = 1e4 
        
        d_x = np.frombuffer(self.data.sma_20.get(size=self.p.state_shape['raw_state'].shape[0])) -\
            np.frombuffer(self.data.sma_8.get(size=self.p.state_shape['raw_state'].shape[0]))
        
        #x[:,0] -= self.raw_state[0,0] 
        #x[:,1] += self.raw_state[0,0]
        #d_x =  x * 1e2
        

        self.state['raw_state'] = self.raw_state
        self.state['model_input'] = d_x[:, None] * T
        
        #print('raw_state:', self.state['raw_state'].shape)
        #print('model_input:', self.state['model_input'] .shape)
        
        return self.state
    
    
    def get_state(self):
        T = 1e4 
      
        x = np.stack(
            [
                #self.raw_state[-16:,1],
                #- self.raw_state[-16:,2],
                np.frombuffer(self.data.open.get(size=self.p.state_shape['raw_state'].shape[0])),
                np.frombuffer(self.data.sma_4.get(size=self.p.state_shape['raw_state'].shape[0])), 
                np.frombuffer(self.data.sma_8.get(size=self.p.state_shape['raw_state'].shape[0])), 
                np.frombuffer(self.data.sma_16.get(size=self.p.state_shape['raw_state'].shape[0])), 
                np.frombuffer(self.data.sma_32.get(size=self.p.state_shape['raw_state'].shape[0])),
                np.frombuffer(self.data.sma_64.get(size=self.p.state_shape['raw_state'].shape[0])),
                np.frombuffer(self.data.sma_128.get(size=self.p.state_shape['raw_state'].shape[0])),
                np.frombuffer(self.data.sma_256.get(size=self.p.state_shape['raw_state'].shape[0])), 
            ], 
            axis=-1
        )
        
        d_x = np.gradient(x, axis=1) * T
        
        #x[:,0] -= self.raw_state[0,0] 
        #x[:,1] += self.raw_state[0,0]
        #d_x =  x * 1e2
        

        self.state['raw_state'] = self.raw_state
        self.state['model_input'] = d_x
        
        #print('raw_state:', self.state['raw_state'].shape)
        #print('model_input:', self.state['model_input'] .shape)
        
        return self.state
    
    def get_reward(self):
        """
        Defines reward as function of last closed trade result with penalty for erronious order placement.
        """
        # Low-value term:
        r = (self.broker.get_value() / self.env.broker.startingcash - 1.0) * 10
        
        #r = 0
        
        # Single trade result:
        if self.trade_just_closed:
            r += self.trade_result * (1 + self.iteration * 1e-4) 
            self.trade_just_closed = False
            
        # Penalty for failed order:
        #if self.order_failed:
        #    r -= self.order_penalty
        #    self.order_failed = False
            
        # Final value:
        #if self.is_done:
        #    r = (self.broker.get_value() / self.env.broker.startingcash - 1.0) * 100
        
        
            
        # Time penalty:
        r -= self.iteration * 1e-5

        return r / 40  # since reward is just quick-example, denominator is here to keep gradients sane
    
# Set backtesting engine parameters:

time_embed_dim = 16

state_shape = {
    'raw_state': spaces.Box(low=-100, high=100, shape=(time_embed_dim, 4)),
    'model_input': spaces.Box(low=-100, high=100, shape=(time_embed_dim, 8))
}

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    MyStrategy,
    state_shape=state_shape,
    #portfolio_actions=('close_hold', 'buy_hold', 'sell_hold'),
    portfolio_actions=('hold', 'buy', 'sell'),
    drawdown_call=5, # max to loose, in percent of initial cash
    target_call=8,  # max to win, same
    skip_frame=10,
)

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)

MyCerebro.addanalyzer(bt.analyzers.DrawDown)

MyDataset = BTgymDataset(
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
    filename='../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    #filename=['../examples/data/DAT_ASCII_EURUSD_M1_201703.csv',
    #          '../examples/data/DAT_ASCII_EURUSD_M1_201704.csv'],
    #filename='../examples/data/DAT_ASCII_EURUSD_M1_201704.csv',
    #filename='../examples/data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=1,
    episode_len_hours=2,
    episode_len_minutes=0,
    start_00=False,
    time_gap_hours=6,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=False,
    render_ylabel='Something',
    render_size_episode=(12,8),
    render_size_human=(10, 5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)

# Set tensorflow distributed cluster and a3c configuration:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_test_3',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    policy_class=LSTMPolicy1D,
    #policy_class=LSTMPolicy1dMulti,
    policy_config={'lstm_layers': (256,)},
    rollout_length=20,
    model_beta=0.03,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=10,
    verbose=2
    
)

[2017-09-13 01:42:16,363] ./tmp/a3c_test_3 created.
[2017-09-13 01:42:16,984] Launcher ready.


In [ ]:
launcher.run()

[2017-09-13 01:42:17,961] worker_0 tf.server started.
[2017-09-13 01:42:17,966] parameters_server started.
[2017-09-13 01:42:18,033] making environment.
[2017-09-13 01:42:18,034] worker_0 is data_master: True
[2017-09-13 01:42:20,257] worker_0:envronment ok.
[2017-09-13 01:42:20,260] A3C_0: init() started
[2017-09-13 01:42:22,197] A3C_0: train op defined
[2017-09-13 01:42:22,265] A3C_0: init() done
[2017-09-13 01:42:22,266] worker_0:trainer ok.
[2017-09-13 01:42:22,852] Press `Ctrl-C` to stop training and close launcher.
[2017-09-13 01:42:22,843] worker_1 tf.server started.
[2017-09-13 01:42:22,848] worker_2 tf.server started.
[2017-09-13 01:42:22,853] worker_3 tf.server started.
[2017-09-13 01:42:22,865] making environment.
[2017-09-13 01:42:22,859] making environment.
[2017-09-13 01:42:22,859] worker_4 tf.server started.
[2017-09-13 01:42:22,865] making environment.
[2017-09-13 01:42:22,862] worker_5 tf.server started.
[2017-09-13 01:42:22,868] worker_3 is data_master: False
[2017-09

Press `Ctrl-C` to stop training and close launcher.


[2017-09-13 01:42:23,481] worker_0: connecting to the parameter server... 
[2017-09-13 01:42:23,960] worker_3:envronment ok.
[2017-09-13 01:42:23,963] A3C_3: init() started
[2017-09-13 01:42:24,084] worker_7:envronment ok.
[2017-09-13 01:42:24,088] A3C_7: init() started
[2017-09-13 01:42:24,113] worker_1:envronment ok.
[2017-09-13 01:42:24,117] A3C_1: init() started
[2017-09-13 01:42:24,130] Initializing all parameters.
[2017-09-13 01:42:24,133] worker_2:envronment ok.
[2017-09-13 01:42:24,136] A3C_2: init() started
[2017-09-13 01:42:24,165] worker_4:envronment ok.
[2017-09-13 01:42:24,170] A3C_4: init() started
[2017-09-13 01:42:24,194] worker_5:envronment ok.
[2017-09-13 01:42:24,203] worker_6:envronment ok.
[2017-09-13 01:42:24,209] A3C_5: init() started
[2017-09-13 01:42:24,217] A3C_6: init() started
[2017-09-13 01:42:27,399] A3C_3: train op defined
[2017-09-13 01:42:27,538] A3C_3: init() done
[2017-09-13 01:42:27,540] worker_3:trainer ok.
[2017-09-13 01:42:27,670] A3C_7: train op 

INFO:tensorflow:Starting standard services.


[2017-09-13 01:42:30,204] worker_5: connecting to the parameter server... 
[2017-09-13 01:42:30,200] worker_2: connecting to the parameter server... 
[2017-09-13 01:42:30,196] Starting standard services.
[2017-09-13 01:42:30,235] worker_6: connecting to the parameter server... 


...

[2017-09-13 08:02:30,261] Saving checkpoint to path ./tmp/a3c_test_3/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test_3/train/model.ckpt


[2017-09-13 08:07:30,264] Saving checkpoint to path ./tmp/a3c_test_3/train/model.ckpt


INFO:tensorflow:Saving checkpoint to path ./tmp/a3c_test_3/train/model.ckpt


[2017-09-13 08:12:30,255] Saving checkpoint to path ./tmp/a3c_test_3/train/model.ckpt


In [23]:
env_config.update({'port': 5090, 'data_port': 5089})
env = BTgymEnv(**env_config)

In [24]:
o = env.reset()

In [25]:
o['model_input'].shape

(16, 1)

In [26]:
o,r,d,i = env.step(0)
print(o)

{'raw_state': array([[ 1.100314,  1.100324,  1.100304,  1.100319],
       [ 1.10029 ,  1.1003  ,  1.10028 ,  1.100295],
       [ 1.100267,  1.100277,  1.100257,  1.100272],
       [ 1.100243,  1.100253,  1.100233,  1.100248],
       [ 1.100219,  1.100229,  1.100209,  1.100224],
       [ 1.100195,  1.100205,  1.100185,  1.1002  ],
       [ 1.100171,  1.100181,  1.100161,  1.100176],
       [ 1.100147,  1.100157,  1.100137,  1.100152],
       [ 1.100122,  1.100132,  1.100112,  1.100127],
       [ 1.100098,  1.100108,  1.100088,  1.100103],
       [ 1.100074,  1.100084,  1.100064,  1.100079],
       [ 1.100049,  1.100059,  1.100039,  1.100054],
       [ 1.100025,  1.100035,  1.100015,  1.10003 ],
       [ 1.1     ,  1.10001 ,  1.09999 ,  1.100005],
       [ 1.099975,  1.099985,  1.099965,  1.09998 ],
       [ 1.099951,  1.099961,  1.099941,  1.099956]]), 'model_input': array([[ 1.2595 ],
       [ 1.27725],
       [ 1.2945 ],
       [ 1.31125],
       [ 1.32675],
       [ 1.34225],
       

In [27]:
env.close()

In [ ]:
MyStrategy.inner_embedding

In [ ]:
a = np.arange(10)

In [ ]:
a

In [ ]:
a[-5:]